In [ ]:
print("\n... IMPORTS STARTING ...\n")
print("\n\tVERSION INFORMATION")
# Machine Learning and Data Science Imports
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");
import sklearn; print(f"\t\t– SKLEARN VERSION: {sklearn.__version__}");
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from sklearn.model_selection import GroupKFold, KFold;

!pip install -q keras-cv-attention-models
import keras_cv_attention_models

# Built In Imports
from kaggle_datasets import KaggleDatasets
from multiprocessing import cpu_count
from collections import Counter
from datetime import datetime
from glob import glob
import warnings
import requests
import imageio
import IPython
import sklearn
import urllib
import zipfile
import pickle
import random
import shutil
import string
import math
import time
import gzip
import ast
import sys
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import seaborn as sns
from PIL import Image
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
import plotly
import PIL
import cv2


def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    
print("\n\n... IMPORTS COMPLETE ...\n")
    
print("\n... SEEDING FOR DETERMINISTIC BEHAVIOUR ...\n")
seed_it_all()

In [ ]:
print(f"\n... ACCELERATOR SETUP STARTING ...\n")

# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  
except ValueError:
    TPU = None

if TPU:
    print(f"\n... RUNNING ON TPU - {TPU.master()}...")
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.experimental.TPUStrategy(TPU)
else:
    print(f"\n... RUNNING ON CPU/GPU ...")
    # Yield the default distribution strategy in Tensorflow
    #   --> Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy() 

# What Is a Replica?
#    --> A single Cloud TPU device consists of FOUR chips, each of which has TWO TPU cores. 
#    --> Therefore, for efficient utilization of Cloud TPU, a program should make use of each of the EIGHT (4x2) cores. 
#    --> Each replica is essentially a copy of the training graph that is run on each core and 
#        trains a mini-batch containing 1/8th of the overall batch size
N_REPLICAS = strategy.num_replicas_in_sync
    
print(f"... # OF REPLICAS: {N_REPLICAS} ...\n")

print(f"\n... ACCELERATOR SETUP COMPLTED ...\n")

In [ ]:
print("\n... DATA ACCESS SETUP STARTED ...\n")

if TPU:
    # Google Cloud Dataset path to training and validation images
    DATA_DIR = KaggleDatasets().get_gcs_path('petfinder-pawpularity-score')
    save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
else:
    # Local path to training and validation images
    DATA_DIR = "/kaggle/input/petfinder-pawpularity-score"
    save_locally = None
    
print(f"\n... DATA DIRECTORY PATH IS:\n\t--> {DATA_DIR}")

print(f"\n... IMMEDIATE CONTENTS OF DATA DIRECTORY IS:")
for file in tf.io.gfile.glob(os.path.join(DATA_DIR, "*")): print(f"\t--> {file}")
    
print("\n\n... DATA ACCESS SETUP COMPLETED ...\n")

In [ ]:
print(f"\n... XLA OPTIMIZATIONS STARTING ...\n")

print(f"\n... CONFIGURE JIT (JUST IN TIME) COMPILATION ...\n")
# enable XLA optmizations (10% speedup when using @tf.function calls)
tf.config.optimizer.set_jit(True)

print(f"\n... XLA OPTIMIZATIONS COMPLETED ...\n")

In [ ]:
print("\n... BASIC DATA SETUP STARTING ...\n\n")

print("\n... TRAIN DATAFRAME ..\n")
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
train_df = pd.read_csv(TRAIN_CSV)
train_df["img_path"] = train_df.Id.apply(lambda x: os.path.join(DATA_DIR, "train", x+".jpg"))
train_df["Q"] = pd.qcut(train_df['Pawpularity'], q = 15, labels = range(15))
display(train_df)

print("\n... TEST DATAFRAME ..\n")
TEST_CSV = os.path.join(DATA_DIR, "test.csv")
test_df = pd.read_csv(TEST_CSV)
test_df["img_path"] = test_df.Id.apply(lambda x: os.path.join(DATA_DIR, "test", x+".jpg"))

display(test_df)

print("\n... SAMPLE SUBMISSION DATAFRAME ..\n")
SS_CSV = os.path.join(DATA_DIR, "sample_submission.csv")
ss_df = pd.read_csv(SS_CSV)
display(ss_df)

# Set Other Variables
print("\n... SETTING OTHER VARIABLES ..\n")

INPUT_SHAPE = (240, 240, 3)
N_CLASSES = train_df.Pawpularity.nunique()
REPLICA_BATCH_SIZE = 8
OVERALL_BATCH_SIZE = REPLICA_BATCH_SIZE * N_REPLICAS
AUTO = tf.data.experimental.AUTOTUNE

print("\n\n... BASIC DATA SETUP FINISHING ...\n")

In [ ]:
def flatten_l_o_l(nested_list):
    """ Flatten a list of lists """
    return [item for sublist in nested_list for item in sublist]

def tf_load_image(image, resize_to=INPUT_SHAPE):
    image = tf.image.decode_jpeg(tf.io.read_file(image), channels=INPUT_SHAPE[-1])
    image = tf.image.resize(image, size=resize_to[:-1])
    return image

def rotate_and_crop(images):
    """Rotate the given image with the given rotation degree and crop for the black edges if necessary
    Args:
        image: A `Tensor` representing an image(s) of arbitrary size.
    
    Returns:
        A rotated image.
    """
    
    
    def _largest_rotated_rect(w, h, angle):
        """
        
        Given a rectangle of size wxh that has been rotated by 'angle' (in
        radians), computes the width and height of the largest possible
        axis-aligned rectangle within the rotated rectangle.
        Original JS code by 'Andri' and Magnus Hoff from Stack Overflow
        Converted to Python by Aaron Snoswell
        
        Source: http://stackoverflow.com/questions/16702966/rotate-image-and-crop-out-black-borders
        
        """
        
        quadrant = tf.cast(tf.math.floor(angle / (math.pi/2)), dtype=tf.uint8)
        quadrant = tf.bitwise.bitwise_and(quadrant, tf.constant(3, dtype=quadrant.dtype))
        sign_alpha = tf.cond(tf.bitwise.bitwise_and(quadrant, tf.constant(1, dtype=quadrant.dtype))==tf.constant(0, dtype=quadrant.dtype), lambda: angle, lambda: math.pi-angle)
        alpha = (sign_alpha % math.pi + math.pi) % math.pi

        bb_w = w * tf.math.cos(alpha) + h * tf.math.sin(alpha)
        bb_h = w * tf.math.sin(alpha) + h * tf.math.cos(alpha)

        gamma = tf.cond(w<h, lambda: tf.math.atan2(bb_w, bb_w), lambda: tf.math.atan2(bb_w, bb_w))

        delta = math.pi - alpha - gamma

        length = tf.cond(w<h, lambda: h, lambda: w)

        d = length * tf.math.cos(alpha)
        a = d * tf.math.sin(alpha) / tf.math.sin(delta)

        y = a * tf.math.cos(gamma)
        x = y * tf.math.tan(gamma)

        return (bb_w - 2 * x, bb_h - 2 * y)
  
    # Get desired output dimensions
    output_height, output_width = tf.constant(INPUT_SHAPE[0], dtype=tf.float32), tf.constant(INPUT_SHAPE[1], dtype=tf.float32)

    rotation_degree = (math.pi/180)*tf.random.normal(shape=(), stddev=10)
    images = tfa.image.rotate(images, rotation_degree, interpolation='BILINEAR')

    # Center crop to ommit black noise on the edges
    lrr_width, lrr_height = _largest_rotated_rect(output_height, output_width, rotation_degree)
    lrr_offset_w, lrr_offset_h = tf.cast(tf.math.round((output_width-lrr_width)/2), dtype=tf.int32), tf.cast(tf.math.round((output_height-lrr_height)/2), dtype=tf.int32)
    lrr_width, lrr_height = tf.cast(tf.math.round(lrr_width), dtype=tf.int32), tf.cast(tf.math.round(lrr_height), dtype=tf.int32)

    images = tf.image.crop_to_bounding_box(images, lrr_offset_h, lrr_offset_w, target_height=lrr_height, target_width=lrr_width)
    images = tf.image.resize(images, (output_height, output_width))
    
    return images


def simple_augmentation(images, labels):
    # Random Horizontal Flip
    images = tf.image.random_flip_left_right(images)
    
    # images = rotate_and_crop(images)
    
    images = tfa.image.random_cutout(images, mask_size=(2*(INPUT_SHAPE[0]//30),2*(INPUT_SHAPE[0]//30)))
    
    # Random Saturation
    images = tf.image.random_saturation(images, 0.975, 1.025)

    # Random Hue
    images = tf.image.random_hue(images, 0.0125)
    
    # Random Brightness
    images = tf.image.random_brightness(images, 0.125)
    
    return images, labels

# Simple Augmentation

In [ ]:
PLT_N = 4
plt.figure(figsize=(20,12))
for i in range(PLT_N):
    o_img = tf_load_image(train_df.sample(1).img_path.values[0])
    a_img = simple_augmentation(tf.expand_dims(o_img, axis=0), None)[0][0]
    plt.subplot(2,4,i+1)
    plt.imshow(o_img/255.)
    plt.title("Original Image", fontweight="bold")
    plt.axis(False)
    
    plt.subplot(2,4,i+5)
    plt.imshow(a_img/255.)
    plt.title("Random Augmentation", fontweight="bold")
    plt.axis(False)
    
plt.tight_layout()
plt.show()

# Function To Create Splits

In [ ]:
N_FOLDS = 8
kf = KFold(n_splits=N_FOLDS)

dataset_folds = {}
for i, (train_indices, val_indices) in enumerate(kf.split(X=train_df["img_path"], y=train_df["Pawpularity"])):
    SUB_TRAIN_IMG_PATH_DS = train_df.iloc[train_indices].img_path.values
    SUB_TRAIN_LABEL_DS = train_df.iloc[train_indices].Pawpularity.values.astype(np.float32)
    
    SUB_VAL_IMG_PATH_DS = train_df.iloc[val_indices].img_path.values
    SUB_VAL_LABEL_DS = train_df.iloc[val_indices].Pawpularity.values.astype(np.float32)
    
    sub_train_img_ds = tf.data.Dataset.from_tensor_slices(SUB_TRAIN_IMG_PATH_DS)
    sub_train_img_ds = sub_train_img_ds.map(tf_load_image, num_parallel_calls=tf.data.AUTOTUNE).cache()
    sub_train_lbl_ds = tf.data.Dataset.from_tensor_slices(SUB_TRAIN_LABEL_DS)
    sub_train_ds = tf.data.Dataset.zip((sub_train_img_ds, sub_train_lbl_ds))
    sub_train_ds = sub_train_ds.shuffle((len(train_df)//N_FOLDS)*(N_FOLDS-1)) 
    sub_train_ds = sub_train_ds.batch(OVERALL_BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
    sub_train_ds = sub_train_ds.map(simple_augmentation, num_parallel_calls=tf.data.AUTOTUNE)
    
    sub_val_img_ds = tf.data.Dataset.from_tensor_slices(SUB_VAL_IMG_PATH_DS)
    sub_val_img_ds = sub_val_img_ds.map(tf_load_image, num_parallel_calls=tf.data.AUTOTUNE).cache()
    sub_val_lbl_ds = tf.data.Dataset.from_tensor_slices(SUB_VAL_LABEL_DS)
    sub_val_ds = tf.data.Dataset.zip((sub_val_img_ds, sub_val_lbl_ds))
    sub_val_ds = sub_val_ds.shuffle(len(train_df)//N_FOLDS) 
    sub_val_ds = sub_val_ds.batch(OVERALL_BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

    dataset_folds[f"fold_{i}"] = {"train_ds":sub_train_ds, "val_ds":sub_val_ds}

test_img_ds = tf.data.Dataset.from_tensor_slices(test_df.img_path.values)
test_ds = test_img_ds.map(tf_load_image, num_parallel_calls=tf.data.AUTOTUNE).batch(OVERALL_BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

dataset_folds

# Function To Grab A Keras Model

In [ ]:
def get_keras_cv_model(bb, pp_fn, make_trainable=False, input_shape=INPUT_SHAPE, head_size=64, dropout_rate=0.3, print_summary=True):
    """
    
    Function to get a custom model. This function allows for any model from the 
    keras applications library of models.
    
    Args:
        bb (tf.keras.applications.<model> functional backbone):
            The specific model backbone function to use (from tf.keras API)
        pp_fn (tf.keras.applications.<model>.preprocess_input):
            The specific preprocessing function to use (from tf.keras API)
        make_trainable (optional, bool):
            Whether or not to set the bulk of the model to be trainable
        input_shape(optional, tuple of ints):
            A tuple of integers describing the input shape the model should expect
        head_size(optional, int): 
            The number of nodes in the final head FC layer
        dropout_rate(optional, float): 
            The dropout to be applied between the final head layer and 
            the classification/regression output layer
        print_summary(optional, bool):
            Whether or not to print the model summary
        
    Returns:
        Custom model for petfinder
    
    """
    # Get the backbone
    bb_submodel = bb(include_top=False, weights="imagenet", input_shape=INPUT_SHAPE, pooling="avg")
    
    # Set trainability
    if not make_trainable:
        bb_submodel.trainable=False
    
    # Define functional model
    _inputs = tf.keras.layers.Input(input_shape)
    preprocessed_inputs = pp_fn(_inputs)
    
    x = bb_submodel(preprocessed_inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(head_size, activation='relu')(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    _outputs = tf.keras.layers.Dense(1, activation="relu")(x)
    
    _model = tf.keras.Model(inputs=_inputs, outputs=_outputs)
    if print_summary:
        print(_model.summary())
    return _model

# Try With Fold 0

In [ ]:
N_EPOCHS=25

MODELS_TO_TEST = [
    tf.keras.applications.EfficientNetB1,
    tf.keras.applications.MobileNetV2,
    tf.keras.applications.ResNet50,
    tf.keras.applications.DenseNet121
]
PP_TO_GO_W_TEST = [
    tf.keras.applications.efficientnet.preprocess_input,
    tf.keras.applications.mobilenet_v2.preprocess_input,
    tf.keras.applications.resnet50.preprocess_input,
    tf.keras.applications.densenet.preprocess_input,
]

histories = []
for BB, PP_FN in zip(MODELS_TO_TEST, PP_TO_GO_W_TEST):    
    print(f"\n\n\n... STARTING TRAINING FOR {BB._keras_api_names[-1].upper()} ...\n\n")
    with strategy.scope():
        
        callback_list = [
            tf.keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.8, patience=2, min_lr=1e-9, verbose=1)
        ]
        
        model = get_keras_cv_model(BB, PP_FN)
        model.compile(
            loss="mse",
            optimizer=tf.keras.optimizers.Adam(0.001),
            metrics=[tf.keras.metrics.RootMeanSquaredError(),]
        )
        
        histories.append(model.fit(dataset_folds[f"fold_0"]["train_ds"],
                                   validation_data=dataset_folds[f"fold_0"]["val_ds"],
                                   callbacks=callback_list, epochs=N_EPOCHS,))

In [ ]:
for metric_name in ["loss", "root_mean_squared_error"]:
    fig = go.Figure()
    for name, history, color in zip(["EfficientNetB1", "MobileNetV2", "ResNet50", "DenseNet121"], histories, ["red", "blue", "orange", "green"]):        
        fig.add_trace(go.Scatter(
            x=np.arange(len(history.history[f"{metric_name}"])),
            y=history.history[f"{metric_name}"],
            mode='lines+markers',
            legendgroup=name,
            text=f"<b>Training {name}</b>",
            legendgrouptitle_text=f"<b>{name}</b>",
            visible="legendonly",
            name=f"<b>Training {metric_name.replace('_', ' ').title()}</b>",
            marker=dict(color=f"{color}"),
            line=dict(color=f"{color}")
        ))
        fig.add_trace(go.Scatter(
            x=np.arange(len(history.history[f"{metric_name}"])),
            y=history.history[f"val_{metric_name}"],
            mode='lines+markers',
            legendgroup=name,
            text=f"<b>Validation {name}</b>",
            legendgrouptitle_text=f"<b>{name}</b>",
            name=f"<b>Validation {metric_name.replace('_', ' ').title()}</b>",
            marker=dict(color=f"dark{color}"),
            line=dict(color=f"dark{color}")
        ))
        fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.995),
                          title=f"{name} - {metric_name.replace('_', ' ').title()} Plot",
                          xaxis_title="Epoch Number",
                          yaxis_title="Mean Squared Error" if metric_name=="loss" else "Root Mean Squared Error",
                          legend_title="<b>LEGEND</b>",)
    fig.show()

In [ ]:
for name, history in zip(["EfficientNetB1", "MobileNetV2", "ResNet50", "DenseNet121"], histories):
    for metric_name in ["loss", "root_mean_squared_error"]:
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=np.arange(len(history.history[f"{metric_name}"])),
            y=history.history[f"{metric_name}"],
            mode='lines+markers',
            name=f"<b>Training {metric_name.replace('_', ' ').title()}</b>"
        ))
        fig.add_trace(go.Scatter(
            x=np.arange(len(history.history[f"{metric_name}"])),
            y=history.history[f"val_{metric_name}"],
            mode='lines+markers',
            name=f"<b>Validation {metric_name.replace('_', ' ').title()}</b>"
        ))
        fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.995),
                          title=f"{name} - {metric_name.replace('_', ' ').title()} Plot",
                          xaxis_title="Epoch Number",
                          yaxis_title="Mean Squared Error" if metric_name=="loss" else "Root Mean Squared Error",
                          legend_title="<b>LEGEND</b>",)
        fig.show()

# Try With Another Fold

In [ ]:
N_EPOCHS=25

MODELS_TO_TEST = [
    tf.keras.applications.EfficientNetB1,
    tf.keras.applications.MobileNetV2,
    tf.keras.applications.ResNet50,
    tf.keras.applications.DenseNet121
]
PP_TO_GO_W_TEST = [
    tf.keras.applications.efficientnet.preprocess_input,
    tf.keras.applications.mobilenet_v2.preprocess_input,
    tf.keras.applications.resnet50.preprocess_input,
    tf.keras.applications.densenet.preprocess_input,
]

histories = []
for BB, PP_FN in zip(MODELS_TO_TEST, PP_TO_GO_W_TEST):    
    print(f"\n\n\n... STARTING TRAINING FOR {BB._keras_api_names[-1].upper()} ...\n\n")
    with strategy.scope():
        
        callback_list = [
            tf.keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.8, patience=2, min_lr=1e-9, verbose=1)
        ]
        
        model = get_keras_cv_model(BB, PP_FN)
        model.compile(
            loss="mse",
            optimizer=tf.keras.optimizers.Adam(0.001),
            metrics=[tf.keras.metrics.RootMeanSquaredError(),]
        )
        
        histories.append(model.fit(dataset_folds[f"fold_5"]["train_ds"],
                                   validation_data=dataset_folds[f"fold_5"]["val_ds"],
                                   callbacks=callback_list, epochs=N_EPOCHS,))

In [ ]:
for metric_name in ["loss", "root_mean_squared_error"]:
    fig = go.Figure()
    for name, history, color in zip(["EfficientNetB1", "MobileNetV2", "ResNet50", "DenseNet121"], histories, ["red", "blue", "orange", "green"]):        
        fig.add_trace(go.Scatter(
            x=np.arange(len(history.history[f"{metric_name}"])),
            y=history.history[f"{metric_name}"],
            mode='lines+markers',
            legendgroup=name,
            text=f"<b>Training {name}</b>",
            legendgrouptitle_text=f"<b>{name}</b>",
            visible="legendonly",
            name=f"<b>Training {metric_name.replace('_', ' ').title()}</b>",
            marker=dict(color=f"{color}"),
            line=dict(color=f"{color}")
        ))
        fig.add_trace(go.Scatter(
            x=np.arange(len(history.history[f"{metric_name}"])),
            y=history.history[f"val_{metric_name}"],
            mode='lines+markers',
            legendgroup=name,
            text=f"<b>Validation {name}</b>",
            legendgrouptitle_text=f"<b>{name}</b>",
            name=f"<b>Validation {metric_name.replace('_', ' ').title()}</b>",
            marker=dict(color=f"dark{color}"),
            line=dict(color=f"dark{color}")
        ))
        fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.995),
                          title=f"{name} - {metric_name.replace('_', ' ').title()} Plot",
                          xaxis_title="Epoch Number",
                          yaxis_title="Mean Squared Error" if metric_name=="loss" else "Root Mean Squared Error",
                          legend_title="<b>LEGEND</b>",)
    fig.show()

In [ ]:
for name, history in zip(["EfficientNetB1", "MobileNetV2", "ResNet50", "DenseNet121"], histories):
    for metric_name in ["loss", "root_mean_squared_error"]:
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=np.arange(len(history.history[f"{metric_name}"])),
            y=history.history[f"{metric_name}"],
            mode='lines+markers',
            name=f"<b>Training {metric_name.replace('_', ' ').title()}</b>"
        ))
        fig.add_trace(go.Scatter(
            x=np.arange(len(history.history[f"{metric_name}"])),
            y=history.history[f"val_{metric_name}"],
            mode='lines+markers',
            name=f"<b>Validation {metric_name.replace('_', ' ').title()}</b>"
        ))
        fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.995),
                          title=f"{name} - {metric_name.replace('_', ' ').title()} Plot",
                          xaxis_title="Epoch Number",
                          yaxis_title="Mean Squared Error" if metric_name=="loss" else "Root Mean Squared Error",
                          legend_title="<b>LEGEND</b>",)
        fig.show()